## CSC 369 Project

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('drive/My Drive/CSC369Project/NFLPlayByPlay2009-2018.csv', delimiter=',')
data

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (42,166,167,168,169,174,175,178,179,182,183,188,189,190,191,194,195,203,204,205,218,219,220,231,232,233,238,240,241,249) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,...,assist_tackle_2_player_id,assist_tackle_2_player_name,assist_tackle_2_team,assist_tackle_3_player_id,assist_tackle_3_player_name,assist_tackle_3_team,assist_tackle_4_player_id,assist_tackle_4_player_name,assist_tackle_4_team,pass_defense_1_player_id,pass_defense_1_player_name,pass_defense_2_player_id,pass_defense_2_player_name,fumbled_1_team,fumbled_1_player_id,fumbled_1_player_name,fumbled_2_player_id,fumbled_2_player_name,fumbled_2_team,fumble_recovery_1_team,fumble_recovery_1_yards,fumble_recovery_1_player_id,fumble_recovery_1_player_name,fumble_recovery_2_team,fumble_recovery_2_yards,fumble_recovery_2_player_id,fumble_recovery_2_player_name,return_team,return_yards,penalty_team,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv
0,46,2009091000,PIT,TEN,PIT,home,TEN,TEN,30.0,2009-09-10,900.0,1800.0,3600.0,Half1,0,1,0,1,NaN,0.0,15:00,TEN 30,0,0,R.Bironas kicks 67 yards from TEN 30 to PIT 3....,kickoff,0.0,0,0,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PIT,39.0,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
1,68,2009091000,PIT,TEN,PIT,home,TEN,PIT,58.0,2009-09-10,893.0,1793.0,3593.0,Half1,0,1,0,1,1.0,0.0,14:53,PIT 42,10,5,(14:53) B.Roethlisberger pass short left to H....,pass,5.0,0,0,1.0,0,0,0,short,left,-3.0,8.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
2,92,2009091000,PIT,TEN,PIT,home,TEN,PIT,53.0,2009-09-10,856.0,1756.0,3556.0,Half1,0,1,0,1,2.0,0.0,14:16,PIT 47,5,2,(14:16) W.Parker right end to PIT 44 for -3 ya...,run,-3.0,0,0,0.0,0,0,0,NaN,NaN,NaN,NaN,right,end,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
3,113,2009091000,PIT,TEN,PIT,home,TEN,PIT,56.0,2009-09-10,815.0,1715.0,3515.0,Half1,0,1,0,1,3.0,0.0,13:35,PIT 44,8,2,(13:35) (Shotgun) B.Roethlisberger pass incomp...,pass,0.0,1,0,1.0,0,0,0,deep,right,34.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
4,139,2009091000,PIT,TEN,PIT,home,TEN,PIT,56.0,2009-09-10,807.0,1707.0,3507.0,Half1,0,1,0,1,4.0,0.0,13:27,PIT 44,8,2,(13:27) (Punt formation) D.Sepulveda punts 54 ...,punt,0.0,0,0,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449366,4059,2018121700,CAR,NO,CAR,home,NO,CAR,66.0,2018-12-17,63.0,63.0,63.0,Half2,0,20,0,4,2.0,0.0,01:03:00,CAR 34,10,19,(1:03) (Shotgun) C.Newton pass incomplete shor...,pass,0.0,1,0,1.0,0,0,0,short,middle,12.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00-0029607,D.Davis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
449367,4

Find whether run or pass is better for each down and distance based on EPA of historical plays

In [3]:
epaData = data.query("play_type == 'pass' or play_type == 'run'")[['down', 'ydstogo', 'play_type', 'epa']]
epaData.dropna(inplace=True)
# group all plays with yards to go of > 15 into the 15 group
epaData['ydstogo'] = epaData['ydstogo'].apply(lambda ytg: 15 if ytg > 15 else ytg)
epaData = epaData.groupby(['down', 'ydstogo', 'play_type']).mean()

epaData = epaData.reset_index()
# join the data with itself so run and pass for the same situation can be compared
epaData = epaData.merge(epaData, on=['down', 'ydstogo'])
# select rows where the two play types are not the same
epaData = epaData.query("play_type_x != play_type_y and play_type_x == 'run'")

optimalEpa = []
for index, row in epaData.iterrows():
  selection = [int(row['down']), row['ydstogo']]
  # select the play type with the higher EPA as the optimal play type
  selection.append(row['play_type_x'] if row['epa_x'] > row['epa_y'] else row['play_type_y'])
  selection.append(row['epa_x'] if row['epa_x'] > row['epa_y'] else row['epa_y'])
  optimalEpa.append(selection)

pd.DataFrame(optimalEpa, columns=['down', 'ydstogo', 'optimal_play_type', 'epa'])

,down,ydstogo,optimal_play_type,epa
0,1,1,run,0.014246
1,1,2,pass,0.088037
2,1,3,pass,0.222827
3,1,4,run,0.097081
4,1,5,pass,0.101250
5,1,6,pass,0.211018
6,1,7,pass,0.317807
7,1,8,pass,0.208934
8,1,9,pass,0.205926
9,1,10,pass,0.081499


Streaming + ML model to predict EPA
---

In [4]:
# always set shotgun to zero for punts and field goals it does not apply
data.loc[data.play_type == "field_goal", "shotgun"] = 0
data.loc[data.play_type == "punt", "shotgun"] = 0

# only look at plays of the desired type
modelData = data.query("play_type == 'pass' or play_type == 'run' or play_type == 'punt' or play_type == 'field_goal'")

# select the columns of interest
modelData = modelData[['down','ydstogo', 'play_type', 'shotgun', 'pass_location', 'pass_length', 'run_location', 'run_gap', 'epa']]
print(modelData)
# One Hot Encode the data
modelData = pd.get_dummies(modelData)
modelData.dropna(inplace=True)

indVars = modelData.drop('epa', axis = 1)

depVar = modelData['epa']
print(indVars)

        down  ydstogo play_type  ...  run_location run_gap       epa
1        1.0       10      pass  ...           NaN     NaN  0.077907
2        2.0        5       run  ...         right     end -1.402760
3        3.0        8      pass  ...           NaN     NaN -1.712583
4        4.0        8      punt  ...           NaN     NaN  2.097796
5        1.0       10       run  ...        middle     NaN -0.696302
...      ...      ...       ...  ...           ...     ...       ...
449363   2.0        1      pass  ...           NaN     NaN -0.180769
449364   3.0        1      pass  ...           NaN     NaN  0.253368
449366   2.0       10      pass  ...           NaN     NaN -0.301121
449367   3.0       10      pass  ...           NaN     NaN -0.320799
449368   4.0        5      pass  ...           NaN     NaN -0.286672

[353060 rows x 9 columns]
        down  ydstogo  shotgun  ...  run_gap_end  run_gap_guard  run_gap_tackle
1        1.0       10        0  ...            0              0  

In [5]:
from sklearn.model_selection import train_test_split

# divide the data into testing and training sets
x_train, x_test, y_train, y_test = train_test_split(indVars, depVar, train_size = 0.7, test_size = 0.3, random_state = 100)
x_train

,down,ydstogo,shotgun,play_type_field_goal,play_type_pass,play_type_punt,play_type_run,pass_location_left,pass_location_middle,pass_location_right,pass_length_deep,pass_length_short,run_location_left,run_location_middle,run_location_right,run_gap_end,run_gap_guard,run_gap_tackle
102938,1.0,10,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0
22749,2.0,13,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
57682,2.0,7,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0
392468,2.0,7,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0
73518,4.0,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83716,3.0,5,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0
434159,3.0,9,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
323269,2.0,8,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0
267819,2.0,10,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0


In [6]:
!python -m pip install statsmodels 


In [7]:
import statsmodels.api as sm


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [8]:
# add a constant to the training data
x_train_sm = sm.add_constant(x_train)

# Fitting the regression model using 'OLS'
lr = sm.OLS(y_train, x_train_sm).fit()

# Printing the parameters
lr.params

const                  -0.637913
down                   -0.013293
ydstogo                -0.007903
shotgun                 0.018804
play_type_field_goal    0.759400
play_type_pass         -0.973004
play_type_punt          0.797771
play_type_run          -1.222079
pass_location_left      0.735702
pass_location_middle    0.900072
pass_location_right     0.703026
pass_length_deep        1.289637
pass_length_short       1.049163
run_location_left       1.231200
run_location_middle     1.880564
run_location_right      1.209913
run_gap_end             0.764857
run_gap_guard           0.657982
run_gap_tackle          0.651347
dtype: float64

In [9]:
# View stats about the model
lr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    epa   R-squared:                       0.067
Model:                            OLS   Adj. R-squared:                  0.067
Method:                 Least Squares   F-statistic:                     1111.
Date:                Sat, 11 Dec 2021   Prob (F-statistic):               0.00
Time:                        02:12:36   Log-Likelihood:            -4.2677e+05
No. Observations:              246042   AIC:                         8.536e+05
Df Residuals:                  246025   BIC:                         8.538e+05
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.6379      0.016    -40.325      0.000      -0.669      -0.607
down                    -0.0133      0.004     -3.380      0.001      -0.021      -0.006
ydstogo                 -0.0079      0.001    -11.025      0.000      -0.009      -0.006
shotgun                  0.0188      0.007      2.836      0.005       0.006       0.032
play_type_field_goal     0.7594      0.018     41.985      0.000       0.724       0.795
play_type_pass          -0.9730      0.016    -60.129      0.000      -1.005      -0.941
play_type_punt           0.7978      0.016     51.215      0.000       0.767       0.828
play_type_run           -1.2221      0.039    -31.211      0.000      -1.299      -1.145
pass_location_left       0.7357      0.008     89.313      0.000       0.720       0.752
pass_location_middle     0.9001      0.009     99.512      0.000       0.882       0.918
pass_location_right      0.7030      0.008     86.710      0.000       0.687       0.719
pass_length_deep         1.2896      0.012    111.733      0.000       1.267       1.312
pass_length_short        1.0492      0.010    108.086      0.000       1.030       1.068
run_location_left        1.2312      0.397      3.100      0.002       0.453       2.010
run_location_middle      1.8806      0.049     38.205      0.000       1.784       1.977
run_location_right       1.2099      0.397      3.046      0.002       0.431       1.988
run_gap_end              0.7649      0.396      1.931      0.054      -0.012       1.541
run_gap_guard            0.6580      0.396      1.660      0.097      -0.119       1.435
run_gap_tackle           0.6513      0.396      1.644      0.100      -0.125       1.428
==============================================================================
Omnibus:                    36501.042   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           366447.640
Skew:                          -0.395   Prob(JB):                         0.00
Kurtosis:                       8.926   Cond. No.                     2.42e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 4.02e-26. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [10]:
!python -m pip install pyspark 


In [11]:
from pyspark import SparkConf
from pyspark.context import SparkContext

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

In [12]:
# given the down and distance of a play, generate all of the potential plays that could be run
def generatePotentialPlays(sc, downDist):
  possiblePlays = []
  baseArr = [0] * 4
  for i in range(4): # 4 different play types
    playData = baseArr.copy()
    playData[i] = 1 # set the play type
    if i == 0 or i == 2: # field goal or punt
      playData.insert(0, 0)  # shotgun always 0 for field goal and punt since not applicable
       # add 11 zeroes to the end of array for the run and pass options since they don't apply
      playData = np.pad(playData, (0, 11), 'constant')
      possiblePlays.append(playData)

    elif i == 1: # pass
      # for each pass location
      for j in range(3): 
        passData = np.pad(playData, (0, 3))
        passData[-(j + 1)] = 1 # set the pass location
        # for each pass length
        for k in range(2):
          specificPassPlay = np.pad(passData, (0, 2))
          specificPassPlay[-(k + 1)] = 1 # set the pass length
          # add zeroes for all of the run columns and 1 to the front for shotgun
          specificPassPlay = np.pad(specificPassPlay, (1, 6)) 
          underCenter = specificPassPlay.copy()
          possiblePlays.append(underCenter)

          # make a copy of the play being run from shotgun
          shotgun = specificPassPlay.copy()
          shotgun[0] = 1
          possiblePlays.append(shotgun)

    elif i == 3: # run
      # for each run location
      for j in range(3):
        # add zeroes for each location and set all the pass play cols to zero as well
        runData = np.pad(playData, (0, 8))
        runData[-(j + 1)] = 1 # set the run location
        # run middle keep gaps all zeroes since middle is the gap as well
        if j == 1: 
          specificRunPlay = np.pad(runData, (1, 3)) # adding 1 zero to front for shotgun
          underCenter = specificRunPlay.copy()
          possiblePlays.append(underCenter)

          # make a copy of the play being run from shotgun
          shotgun = specificRunPlay.copy()
          shotgun[0] = 1
          possiblePlays.append(shotgun)

        else:
          # for each run gap
          for k in range(3):
            specificRunPlay = np.pad(runData, (1, 3)) # adding 1 zero to front for shotgun
            specificRunPlay[-(k + 1)] = 1 # set the run gap
          
            underCenter = specificRunPlay.copy()
            possiblePlays.append(underCenter)

            # make a copy of the play being run from shotgun
            shotgun = specificRunPlay.copy()
            shotgun[0] = 1
            possiblePlays.append(shotgun)

  # Add the down and distance to the front of the play data for each play
  for i, play in enumerate(possiblePlays):
    possiblePlays[i] = np.insert(play, 0, downDist[1]) # distance
    possiblePlays[i] = np.insert(possiblePlays[i], 0, downDist[0]) # down

  # add the constant
  possiblePlaysWConst = sm.add_constant(possiblePlays, has_constant='add')
  # make the plays into an RDD
  return sc.parallelize(possiblePlaysWConst)



In [13]:
downDist = [3, 5]
possiblePlays = generatePotentialPlays(sc, downDist)
# number of potential plays
len(possiblePlays.collect())

28

In [14]:
# game used to test the program
testGame = data.query("game_id == 2018121700")
# only look at desired plays
testGame = testGame.query("play_type == 'pass' or play_type == 'run' or play_type == 'punt' or play_type == 'field_goal'")

testGame = testGame[['down','ydstogo']] 

# for each play, generate the set of potential play calls and add it to the list
playOptionsForGame = []
for i in range(len(testGame.index)):
  playOptionsForGame.append(generatePotentialPlays(sc, testGame.iloc[i]))

# number of plays in the test game
len(playOptionsForGame)


136

In [18]:
from pyspark.streaming import StreamingContext
import traceback
import time

ssc = StreamingContext(sc, 1)
# create a stream with the list of plays for the game and process them 1 at a time
plays = ssc.queueStream(playOptionsForGame, True)

# Convert the numeric data into a description of the play
def generatePlayString(binPlayData):
  playDesc = ''  
  if binPlayData[0]:
    playDesc += 'shotgun '

  if binPlayData[1]:
    playDesc += 'field goal '

  if binPlayData[2]:
    playDesc += 'pass '

  if binPlayData[3]:
    playDesc += 'punt '

  if binPlayData[4]:
    playDesc += 'run '

  if binPlayData[5]:
    playDesc += 'left '

  if binPlayData[6]:
    playDesc += 'middle '

  if binPlayData[7]:
    playDesc += 'right '

  if binPlayData[8]:
    playDesc += 'deep '

  if binPlayData[9]:
    playDesc += 'short '

  if binPlayData[10]:
    playDesc += 'left '

  if binPlayData[11]:
    playDesc += 'middle '

  if binPlayData[12]:
    playDesc += 'right '

  if binPlayData[13]:
    playDesc += 'end '

  if binPlayData[14]:
    playDesc += 'guard '

  if binPlayData[15]:
    playDesc += 'tackle '

  return playDesc.strip()
    

def evaluatePlay(binPlayData):
  # predict the play's EPA using the model
  epa = lr.predict(binPlayData)[0]
  # Get a text description of the play from the data
  playDesc = generatePlayString(binPlayData[3:])
  return (playDesc, epa)

# Make an EPA prediction for each play in the RDD show the 5 plays with the 
# highest predicted EPA
def findBestPlay(rdd):
    start = time.time()
    if rdd.isEmpty():
        return
    try:
      play = rdd.first()
      down, dist = [play[1], play[2]]
      print('Down: ' + str(down))
      print('Distance: ' + str(dist))
      # Map the rdd to evaluatePlay and from this func return a tuple with a string describing the play and its predicted EPA
      plays = rdd.map(lambda play: evaluatePlay(play))
      print('Top five plays: ')
      # sort the plays by EPA in descending order and select the top 5
      print(sorted(plays.collect(), key=lambda x: x[1], reverse=True)[:6])
      end = time.time()
      print(f"time required to make predictions: {end - start} seconds")
      print()

    except Exception:
        print(traceback.format_exc())

# find the best play for each of the situations in the test game
plays.foreachRDD(findBestPlay)

ssc.start()
# run for 30 seconds
ssc.awaitTerminationOrTimeout(30)
ssc.stop(stopSparkContext=False)

# averages right around two seconds to process a situation


Down: 1.0
Distance: 10.0
Top five plays: 
[('shotgun pass middle deep', 0.5052749045738939), ('pass middle deep', 0.48647049929226494), ('shotgun pass left deep', 0.3409043837202723), ('pass left deep', 0.32209997843864335), ('shotgun pass right deep', 0.308228377206465), ('pass right deep', 0.28942397192483604)]
time required to make predictions: 1.9816200733184814 seconds

Down: 2.0
Distance: 6.0
Top five plays: 
[('shotgun pass middle deep', 0.5235935730708755), ('pass middle deep', 0.5047891677892465), ('shotgun pass left deep', 0.3592230522172538), ('pass left deep', 0.34041864693562496), ('shotgun pass right deep', 0.3265470457034465), ('pass right deep', 0.30774264042181765)]
time required to make predictions: 2.093118190765381 seconds

Down: 1.0
Distance: 10.0
Top five plays: 
[('shotgun pass middle deep', 0.5052749045738939), ('pass middle deep', 0.48647049929226494), ('shotgun pass left deep', 0.3409043837202723), ('pass left deep', 0.32209997843864335), ('shotgun pass right 